#### KoAlpaca-Polyglat-12.8B 추론
```
1. github: https://github.com/Beomi/KoAlpaca/tree/main/webui/app.py 
2. parameter data type: touch.float16
3. qutanization: not quantized
4. processor: cpu
5. write type: safetensors
6. transformers: AutoModelForCausalLM, pipeline
```

In [ ]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [ ]:
# GPU 검사
if torch.cuda.is_available():
    print("CUDA is available. GPU can be used.")
else:
    print("CUDA is not available. Using CPU instead.")
    
    
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 사용되지 않는 캐시된 메모리 해제
    print("GPU cache is emptied.")

In [ ]:
# KoAlpaca-ployglot 12.8B (Non 양자화 모델)
# 파일형식: Hugging Transformers 의 Safe serialization 모델
# 파일크기: 1G X 28개
# device_map="auto" 로 설정하면  "cuda" 가 가능하면 "gpu"로 올리게 된다.
# 메모리 부족으로 모델을 cpu 로 변경(device_map 제거)
model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bsf"

#model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
)

In [ ]:
# 로딩 모델 위치
next(model.parameters()).device

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 전체 파라메터 확인
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_parameters}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
def isUsingGpu():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available. Using GPU.")
        return True, device
    else:
        device = torch.device("cpu")
        print("GPU is not available. Using CPU.")
        return False, device
    
def is_model_on_gpu(model):
    return next(model.parameters()).is_cuda    

# 장치 확인   
using_cuda, device = isUsingGpu()
on_gpu_model = is_model_on_gpu(model)

print(f"GPU Using: {using_cuda}, Model GPU Loaded: {on_gpu_model}")
# GPU 가 사용가능하고, 모델이 CPU에 로딩되어 있다면
# 모델을 GPU 로 이동한다.
if(using_cuda == True and on_gpu_model == False):
    model.to(device)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [ ]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)